In [10]:
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
ins = pd.read_csv('InstallmentsTable.csv')
ins.set_index(['borrower_id', 'loan_id'], inplace=True)

## Check Which Loans Have Defaulted  (Miss n installments In a row)

In [3]:
#n = number of installments in a row that you must miss in a row before a default
#WARNING, this should probably be different depending on your installment period
n = 9
#Mask for maximum lag
lag_mask = ins.groupby(level=1).shift(n).due_date.isnull()

#Mask for defaults (must occur beyond lag window if at beginning of installment series)
default_mask = ins.paid_amount.isnull()
for i in range(1, n):
    default_mask = default_mask & ins.groupby(level=1).shift(i).paid_amount.isnull()
    
default_mask = default_mask & ~lag_mask

## Calculate Default Date

In [18]:
ins['due_date_previous'] = ins.groupby(level=1).shift(1)['due_date']
ins.due_date = pd.to_datetime(ins.due_date)
ins.due_date_previous = pd.to_datetime(ins.due_date_previous)
ins['installment_interval'] = (ins.due_date.dt.date - ins.due_date_previous.dt.date).dt.days

In [19]:
defaults = ins[default_mask].groupby(level = ['borrower_id', 'loan_id']).agg({'due_date':'min'})
defaults.columns = ['default_date']

In [27]:
targets = (ins.
join(defaults[['default_date']]).
groupby(level = ['borrower_id', 'loan_id']).
agg({'default_date':{'default_date': 'min', 'default_flag':lambda x: max(~x.isnull())}, 
    'installment_interval':{'installment_peroid_min':'min', 'installment_peroid_max':'max', 'installment_peroid_median':'median'}}))



targets.columns = targets.columns.droplevel()
targets.default_date = pd.to_datetime(targets.default_date)

targets.head(10)

default_date default_flag  installment_peroid_min  \
borrower_id loan_id                                                            
129         1                       NaT        False                      28   
            58      2013-02-04 10:38:55         True                      28   
130         2                       NaT        False                      28   
            59      2011-10-22 07:57:19         True                      28   
131         3                       NaT        False                      28   
            60      2011-10-22 07:58:00         True                      28   
146         4                       NaT        False                      30   
            69                      NaT        False                      28   
            408                     NaT        False                      28   
            4393                    NaT        False                      28   

                     installment_peroid_median  installment_peroid_max  
borrower_id loan_id                                                     
129         1                               31                      31  
            58                              31                     273  
130         2                               31                      31  
            59                              31                      31  
131         3                               31                      31  
            60                              31                      31  
146         4                               31                      31  
            69                              31                     122  
            408                             31                      91  
            4393                            31                      32

In [7]:
targets.to_csv('defaults.csv')

In [29]:
#check calculation
#ins[ins.index.get_level_values('borrower_id') == 129][['due_date', 'paid_amount']]

## Check lags between installments

In [17]:
targets.installment_peroid_median.value_counts().sort_index()

7.0      30125
7.5          2
8.0          3
10.5        12
11.0         1
14.0        12
17.5         5
21.0         2
24.5         2
28.0        19
29.0        10
29.5        30
30.0      1189
30.5      1409
31.0      3541
31.5         1
32.0         1
35.0         2
42.0         1
45.0         1
45.5         2
46.0         4
49.0         1
60.0         2
61.0         1
61.5         2
76.0         2
245.0        1
Name: installment_peroid_median, dtype: int64